# Splits Zadanie - Rozwiązanie
Notebook prezentuje podziały danych i ewaluację modelu zgodnie z dobrymi praktykami ML.


In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import (train_test_split, StratifiedShuffleSplit, KFold, StratifiedKFold, LeaveOneOut, cross_val_score)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [ ]:
# Generowanie danychpip install scikit-learn
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=42)

In [ ]:
# Definicja pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(random_state=42, max_iter=1000))])

## Holdout Split

**Opis teoretyczny:**  
Metoda Holdout polega na jednorazowym podziale danych na zbiór treningowy i testowy.  
Dzięki temu sprawdzamy, jak model generalizuje na niewidziane wcześniej dane.

**Parametry najczęściej spotykane w scikit-learn:**  
- `test_size` – odsetek danych na test (np. 0.3)  
- `random_state` – ziarno losowania dla powtarzalności

**Zalety:**  
- bardzo prosta i szybka implementacja  

**Wady:**  
- ocena może silnie zależeć od konkretnego losowania  
- marnuje dane (nie używamy ich w treningu)  

In [6]:
# 1. Holdout split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
pipeline.fit(X_train, y_train)
print(f"Holdout split accuracy: {pipeline.score(X_test, y_test):.3f}")


Holdout split accuracy: 0.887


## Stratified Shuffle Split

**Opis teoretyczny:**  
To Holdout przeprowadzony wielokrotnie (n _splits_), z zachowaniem proporcji klas (stratą).  
W każdej iteracji wybieramy losowo nowe próbki, ale dbamy, by w zbiorze testowym i treningowym było tyle samo procentowo każdej klasy.

**Kiedy używać:**  
- przy nierównomiernym rozkładzie klas  
- gdy zależy nam na stabilniejszej ocenie niż w pojedynczym Holdout

**Zalety:**  
- mniejsza wariancja wyniku  
- zachowanie rozkładu klas w każdym podziale  

In [7]:
# 2. Stratified Shuffle Split CV
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
scores_sss = cross_val_score(pipeline, X, y, cv=sss, scoring='accuracy')
print(f"StratifiedShuffleSplit CV accuracy: {scores_sss.mean():.3f} ± {scores_sss.std():.3f}")


StratifiedShuffleSplit CV accuracy: 0.898 ± 0.008


## K-Fold Cross-Validation

**Opis teoretyczny:**  
Dzielimy zbiór na K równych części („foldów”). Model uczymy K razy, za każdym razem:
1. Jako walidacyjny używamy innego folda.  
2. Pozostałe K–1 foldów łączymy w zbiór treningowy.  

Średnia metryka z wszystkich uruchomień to końcowa ocena.

**Zalety:**  
- pełne wykorzystanie danych (każdy punkt raz trafia do testu)  
- stabilna ocena  

**Wady:**  
- K-krotnie większy koszt obliczeniowy  

In [8]:
# 3. K-Fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores_kf = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy')
print(f"KFold CV accuracy: {scores_kf.mean():.3f} ± {scores_kf.std():.3f}")


KFold CV accuracy: 0.901 ± 0.012


## Stratified K-Fold Cross-Validation

**Opis teoretyczny:**  
To K-Fold z dodatkowym zachowaniem proporcji klas w każdym foldzie.  
Dzięki temu przy problemach klasyfikacyjnych unikamy sytuacji, że w jakimś foldzie zabraknie przykładów danej klasy.

**Zalety:**  
- łączy stabilność K-Folda z poprawnym rozkładem klas  

In [9]:
# 4. Stratified K-Fold CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_skf = cross_val_score(pipeline, X, y, cv=skf, scoring='accuracy')
print(f"StratifiedKFold CV accuracy: {scores_skf.mean():.3f} ± {scores_skf.std():.3f}")


StratifiedKFold CV accuracy: 0.902 ± 0.028


## Leave-One-Out Cross-Validation (LOO)

**Opis teoretyczny:**  
Specjalny przypadek K-Fold, gdzie K = liczba próbek.  
Dla każdej próbki:
1. Uczymy na wszystkich pozostałych n–1 próbkach.  
2. Testujemy na tej jednej.

**Zalety:**  
- maksymalne wykorzystanie danych  
- minimalny bias  

**Wady:**  
- ekstremalnie kosztowne obliczeniowo przy większych zbiorach  

In [10]:
# 5. Leave-One-Out CV
loo = LeaveOneOut()
scores_loo = cross_val_score(pipeline, X, y, cv=loo, scoring='accuracy', n_jobs=-1)
print(f"LeaveOneOut CV accuracy: {scores_loo.mean():.3f}")


LeaveOneOut CV accuracy: 0.903
